In [0]:
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 116.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

2. Configure Kaggle Credentials

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "veer"
os.environ["KAGGLE_KEY"] = "KGAT_2f4f2c97ddabce143f853dd15bb24796"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS workspace.ecommerce""")

DataFrame[]

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

In [0]:
%sh

cd /Volumes/workspace/ecommerce/ecommerce_data

kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:29<00:00, 158MB/s]


In [0]:
%sh

cd /Volumes/workspace/ecommerce/ecommerce_data

unzip -o ecommerce-behavior-data-from-multi-category-store.zip

ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-2f3bf599-14a3-4d6c-b671-1f nogroup 8.4G Jan 15 07:41 2019-Nov.csv
-rwxrwxrwx 1 spark-2f3bf599-14a3-4d6c-b671-1f nogroup 5.3G Jan 15 07:43 2019-Oct.csv
-rwxrwxrwx 1 spark-2f3bf599-14a3-4d6c-b671-1f nogroup 4.3G Jan 15 07:37 ecommerce-behavior-data-from-multi-category-store.zip


In [0]:
%sh

cd /Volumes/workspace/ecommerce/ecommerce_data

rm -f ecommerce-behavior-data-from-multi-category-store.zip

ls -lh

total 14G
-rwxrwxrwx 1 spark-2f3bf599-14a3-4d6c-b671-1f nogroup 8.4G Jan 15 07:41 2019-Nov.csv
-rwxrwxrwx 1 spark-2f3bf599-14a3-4d6c-b671-1f nogroup 5.3G Jan 15 07:43 2019-Oct.csv


In [0]:
%restart_python

In [0]:
oct_df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
                        header=True,       # Uses first row as column names
                        inferSchema=True   # Automatically detects int, double, timestamp)
)

In [0]:
nov_df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
                        header=True,       # Uses first row as column names
                        inferSchema=True   # Automatically detects int, double, timestamp)
)

In [0]:
print(f"October 2019 - Total Events: {oct_df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
oct_df.printSchema()

October 2019 - Total Events: 42,448,764

SCHEMA:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
print(f"November 2019 - Total Events: {nov_df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
nov_df.printSchema()

November 2019 - Total Events: 67,501,979

SCHEMA:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
oct_df.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |NULL    |543.1  |519107250|566511c2-e2e3-422b-b695-c

In [0]:
# Combined dataset
combined_df = oct_df.union(nov_df)
print(f"\n✅ Combined Dataset - Total Events: {combined_df.count():,}")
print(f"   Date Range: October 2019 - November 2019")


✅ Combined Dataset - Total Events: 109,950,743
   Date Range: October 2019 - November 2019


In [0]:
print(f"Combined 2019 - Total Events: {combined_df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
combined_df.printSchema()

Combined 2019 - Total Events: 109,950,743

SCHEMA:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
# Define your target path (giving it a specific folder name like 'combined_data')
output_path = "/Volumes/workspace/ecommerce/ecommerce_data/combined_output"

# Write the dataframe to CSV
combined_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(output_path)

print(f"✅ Data saved to: {output_path}")

✅ Data saved to: /Volumes/workspace/ecommerce/ecommerce_data/combined_output


In [0]:
%fs ls /Volumes/workspace/ecommerce/ecommerce_data/combined_output

path,name,size,modificationTime
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/_SUCCESS,_SUCCESS,0,1768463480000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/_committed_7138978783768556378,_committed_7138978783768556378,9903,1768463480000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/_started_7138978783768556378,_started_7138978783768556378,0,1768463378000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/part-00000-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-466-1-c000.csv,part-00000-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-466-1-c000.csv,135127596,1768463379000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/part-00001-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-467-1-c000.csv,part-00001-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-467-1-c000.csv,135125587,1768463379000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/part-00002-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-468-1-c000.csv,part-00002-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-468-1-c000.csv,135129098,1768463378000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/part-00003-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-469-1-c000.csv,part-00003-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-469-1-c000.csv,135125163,1768463378000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/part-00004-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-470-1-c000.csv,part-00004-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-470-1-c000.csv,135125092,1768463386000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/part-00005-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-471-1-c000.csv,part-00005-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-471-1-c000.csv,135134875,1768463386000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/combined_output/part-00006-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-472-1-c000.csv,part-00006-tid-7138978783768556378-32cd0ec5-7dc6-4cca-bbf6-395aef6d4b8a-472-1-c000.csv,135135841,1768463386000


In [0]:
# 1. Define your base path for processed parquet files
base_path = "/Volumes/workspace/ecommerce/ecommerce_data/processed_data"

# 2. Save October Data
oct_df.write \
    .mode("overwrite") \
    .parquet(f"{base_path}/oct_2019")

# 3. Save November Data
nov_df.write \
    .mode("overwrite") \
    .parquet(f"{base_path}/nov_2019")

# 4. Save Combined Data
combined_df.write \
    .mode("overwrite") \
    .parquet(f"{base_path}/combined_all")

print("✅ All dataframes saved successfully to:", base_path)

✅ All dataframes saved successfully to: /Volumes/workspace/ecommerce/ecommerce_data/processed_data
